# Exploration File

File for exploring the data 

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("/Users/brigitta/Project/Checkins.txt", delimiter = "\t", header=None, 
                 names=["user", "checkin_time", "latitude", "longitude", "location_id"])

In [ ]:
df = pd.read_csv("C:/Users/huawei/OneDrive/Documents/University/project/Checkins.txt", delimiter = "\t", header=None, 
                 names=["user", "checkin_time", "latitude", "longitude", "location_id"])

In [5]:
df.head(10)

,user,checkin_time,latitude,longitude,location_id
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878
5,0,2010-10-12T23:58:03Z,30.261599,-97.758581,15372
6,0,2010-10-12T22:02:11Z,30.267910,-97.749312,21714
7,0,2010-10-12T19:44:40Z,30.269103,-97.749395,420315
8,0,2010-10-12T15:57:20Z,30.281120,-97.745211,153505
9,0,2010-10-12T15:19:03Z,30.269103,-97.749395,420315
